In [ ]:
import os
import os.path as op
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
%matplotlib inline
import seaborn as sns
sns.set_theme(style="darkgrid", color_codes=True)
sns.set(font_scale=1.35, style="ticks") #set styling preferences

In [ ]:
small_stacked_rh_price = sf_stacked_rh_price[['lever_position_price','income_deciles','IDMerged','tripIndex',
                                       'income', 'realized_in_dollar_2023_relative_to_baseline', 
                                       'Realized_INEXUS_in_dollar_2023', 'potential_in_dollar_2023_relative_to_baseline',
                                       'Potential_INEXUS_in_dollar_2023', 'mode_choice_actual_4', 'duration_door_to_door',
                                       'waitTime_no_replanning','waitTime_replanning', 'mode_choice_planned_BEAM',
                                       'mode_choice_actual_BEAM','mode_actual_baseline']]

In [ ]:
# I think we have a column with this name and it can be added in the beginning
small_stacked_rh_price['incomeInThousands'] = small_stacked_rh_price['income']/1000

In [ ]:
person_income = pd.pivot_table(
   small_stacked_rh_price,
   index=['IDMerged'],
   aggfunc={'incomeInThousands': lambda x: ', '.join(set(x.dropna().astype(str)))}).reset_index() 

In [ ]:
person_income['incomeInThousands'] = person_income['incomeInThousands'].astype(float)

In [ ]:
# Add a column of income ranks
twenty_one_ranks = person_income['incomeInThousands'].quantile([0, 0.048, 0.095, 0.143, 0.191, 0.239, 0.287, 0.335, 0.383, 0.431, 0.479,
                                                                0.527, 0.575, 0.623, 0.671, 0.719, 0.767, 0.815, 0.863, 0.911, 0.959,
                                                                1]).tolist()

In [ ]:
# Add incomeInThousands twenty_one_ranks
conditions  = [(person_income['incomeInThousands'] >= twenty_one_ranks[0]) & (person_income['incomeInThousands'] < twenty_one_ranks[1]), 
               (person_income['incomeInThousands'] >= twenty_one_ranks[1]) & (person_income['incomeInThousands'] < twenty_one_ranks[2]),
               (person_income['incomeInThousands'] >=  twenty_one_ranks[2]) & (person_income['incomeInThousands'] < twenty_one_ranks[3]),
               (person_income['incomeInThousands'] >= twenty_one_ranks[3]) & (person_income['incomeInThousands'] < twenty_one_ranks[4]), 
               (person_income['incomeInThousands'] >=  twenty_one_ranks[4]) & (person_income['incomeInThousands'] < twenty_one_ranks[5]),
               (person_income['incomeInThousands'] >=  twenty_one_ranks[5]) & (person_income['incomeInThousands'] < twenty_one_ranks[6]),
               (person_income['incomeInThousands'] >=  twenty_one_ranks[6]) & (person_income['incomeInThousands'] < twenty_one_ranks[7]),
               (person_income['incomeInThousands'] >=  twenty_one_ranks[7]) & (person_income['incomeInThousands'] < twenty_one_ranks[8]),
               (person_income['incomeInThousands'] >=  twenty_one_ranks[8]) & (person_income['incomeInThousands'] < twenty_one_ranks[9]),
               (person_income['incomeInThousands'] >=  twenty_one_ranks[9]) & (person_income['incomeInThousands'] <= twenty_one_ranks[10]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[10]) & (person_income['incomeInThousands'] <= twenty_one_ranks[11]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[11]) & (person_income['incomeInThousands'] <= twenty_one_ranks[12]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[12]) & (person_income['incomeInThousands'] <= twenty_one_ranks[13]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[13]) & (person_income['incomeInThousands'] <= twenty_one_ranks[14]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[14]) & (person_income['incomeInThousands'] <= twenty_one_ranks[15]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[15]) & (person_income['incomeInThousands'] <= twenty_one_ranks[16]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[16]) & (person_income['incomeInThousands'] <= twenty_one_ranks[17]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[17]) & (person_income['incomeInThousands'] <= twenty_one_ranks[18]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[18]) & (person_income['incomeInThousands'] <= twenty_one_ranks[19]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[19]) & (person_income['incomeInThousands'] <= twenty_one_ranks[20]),
              (person_income['incomeInThousands'] >=  twenty_one_ranks[20]) & (person_income['incomeInThousands'] <= twenty_one_ranks[21])]


choices = [0, 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]

In [ ]:
person_income['incRank'] = np.select(conditions, choices, default=None)

In [ ]:
smaller_stacked_rh_price = pd.merge(left = small_stacked_rh_price, right = person_income, how='left', on = ['IDMerged'], suffixes=('', '_drop'))
smaller_stacked_rh_price.drop([col for col in smaller_stacked_rh_price.columns if 'drop' in col], axis=1, inplace=True)

In [ ]:
lorenz_data_median = pd.pivot_table(
   smaller_stacked_rh_price,
   index=['lever_position_price', 'incRank'],
   aggfunc={'incomeInThousands': np.mean,
           'Potential_INEXUS_in_dollar_2023': np.median,
           'Realized_INEXUS_in_dollar_2023': np.median,
           'realized_in_dollar_2023_relative_to_baseline': np.median,
           'potential_in_dollar_2023_relative_to_baseline': np.median}).reset_index() 

In [ ]:
lorenz_data = pd.pivot_table(
   smaller_stacked_rh_price,
   index=['lever_position_price', 'incRank'],
   aggfunc={'incomeInThousands': np.mean,
           'Potential_INEXUS_in_dollar_2023': np.mean,
           'Realized_INEXUS_in_dollar_2023': np.mean,
           'realized_in_dollar_2023_relative_to_baseline': np.mean,
           'potential_in_dollar_2023_relative_to_baseline': np.mean}).reset_index() 

In [ ]:
palette = {
    8: '#521e4d',
    5: '#8e1d5b',
    3: '#ad1759',
    1.75: '#ca1a50',
    1: '#e13243',
    0.5: '#ee543f',
    0.25: '#f37751',
    0.125: '#f5966d',
    0: '#f7d0b5'}

In [ ]:
lorenz_data['incRank_percent'] = (lorenz_data['incRank'])*5

In [ ]:
import matplotlib.ticker as ticker
sns.set(rc={'figure.figsize':(20,14)})
sns.set_theme(style="whitegrid")
fig, ax = plt.subplots()
ax = sns.lineplot(data = lorenz_data, x = 'incRank_percent', 
                  y = 'potential_in_dollar_2023_relative_to_baseline', hue = 'lever_position_price', palette = palette, ci=None, linewidth = 6)
ax.set_xlabel('Income Rank (Percentiles of Income)', fontsize = 36) #45
ax.set_ylabel('Potential INEXUS ($) relative to Baseline', fontsize = 36) #45
ax.yaxis.set_tick_params(labelsize = 26) #35
ax.xaxis.set_tick_params(labelsize = 26) #35
ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'{int(x)}%'))
plt.legend(labels = ['0% Price', '12.5% Price', '25% Price','50% Price','Baseline','175% Price','300% Price', '500% Price','800% Price'],
         fontsize='medium', fancybox=False, bbox_to_anchor=(1, 0.8), ncol=1, frameon=True, loc = 'best', prop={'size': 26}) #35
plt.savefig('lorenz.svg', format='svg', bbox_extra_artists=(ax.legend(['0% Price', '12.5% Price', '25% Price','50% Price','Baseline','175% Price','300% Price', '500% Price','800% Price'], fancybox=False, bbox_to_anchor=(1, 0.8), fontsize=26),), bbox_inches='tight') #30
plt.savefig('lorenz.png', dpi=300)